In [ ]:
%autosave 60

Autosaving every 60 seconds


In [ ]:
!unzip teta-nn-1-2025.zip

unzip:  cannot find or open teta-nn-1-2025.zip, teta-nn-1-2025.zip.zip or teta-nn-1-2025.zip.ZIP.


In [ ]:
pip install razdel

In [ ]:
pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 53.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
pip install WordCloud

In [ ]:
pip install datasets

In [ ]:
pip install graphviz

In [ ]:
# --- Системные и общие ---
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

# --- SKlearn ---
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# --- NLP-инструменты ---
import nltk
from nltk.corpus import stopwords
from razdel import tokenize as razdel_tokenize
import pymorphy3
from wordcloud import WordCloud

# --- PyTorch & Transformers ---
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from datasets import Dataset as HFDataset

# --- Визуализация ---
from graphviz import Digraph

# --- Константы и настройка среды ---
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

# Определяем устройство: MPS (Apple Silicon GPU), CUDA или CPU
if torch.backends.mps.is_available():
    device = torch.device("mps")
    torch.mps.manual_seed(RANDOM_STATE)
elif torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(RANDOM_STATE)
else:
    device = torch.device("cpu")

print(f"✅ Используемое устройство: {device}")

# --- Инициализация лидерборда ---
leaderboard = pd.DataFrame(columns=['Метод', 'R2_Score'])

ModuleNotFoundError: No module named 'pymorphy3'

In [ ]:
full_train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')



# Разбиваем исходный train на обучающую и валидационную выборки
train_df, val_df = train_test_split(
    full_train_df,
    test_size=0.25,
    random_state=RANDOM_STATE
)

# Сбрасываем индексы для удобства
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Создаем единое текстовое поле для анализа
text_cols = ['title', 'location', 'company', 'skills', 'description']
for df in [train_df, val_df, test_df]:
    # Заполняем пропуски пустыми строками, чтобы избежать ошибок
    df[text_cols] = df[text_cols].fillna('')
    # Объединяем текстовые колонки через разделитель
    df['full_text'] = df[text_cols].agg(' | '.join, axis=1)

print("Пример данных из train:")
display(train_df[['full_text', 'log_salary_from']].head(3))

Пример данных из train:


,full_text,log_salary_from
0,QA Engineer | Москва (м. Сколково / м. Крылатс...,5.298317
1,Senior Backend QA engineer | Санкт-Петербург |...,5.010635
2,Python разработчик (Django) | Москва | Hammer ...,3.912023


## Предобработка текста


In [ ]:
# Загружаем стоп-слова
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('russian'))
# Добавим специфичные для вакансий стоп-слова
stop_words.update(['наш', 'компания', 'команда', 'работа', 'вакансия', 'искать'])

# Инициализируем лемматизатор pymorphy3
morph = pymorphy3.MorphAnalyzer()

def preprocess_text(text: str) -> str:
    """Полный цикл предобработки текста с использованием pymorphy3."""
    # 1. Токенизация
    tokens = [token.text for token in razdel_tokenize(text)]
    # 2. Приведение к нижнему регистру и удаление не-буквенных токенов
    lower_tokens = [token.lower() for token in tokens if token.isalpha()]
    # 3. Лемматизация и удаление стоп-слов
    lemmas = [morph.parse(token)[0].normal_form for token in lower_tokens]
    cleaned_lemmas = [lemma for lemma in lemmas if lemma not in stop_words]
    return " ".join(cleaned_lemmas)

# Демонстрируем работу на одном примере
sample_text = train_df['full_text'].iloc[0]
print(f"ИСХОДНЫЙ ТЕКСТ: '{sample_text[:200]}...'")
print(f"ОБРАБОТАННЫЙ ТЕКСТ: '{preprocess_text(sample_text)}'")

# Применяем ко всем данным
tqdm.pandas(desc="Preprocessing Train")
train_df['processed_text'] = train_df['full_text'].progress_apply(preprocess_text)
tqdm.pandas(desc="Preprocessing Val")
val_df['processed_text'] = val_df['full_text'].progress_apply(preprocess_text)

ИСХОДНЫЙ ТЕКСТ: 'QA Engineer | Москва (м. Сколково / м. Крылатское) | getmatch agency | Python, Java, Go, C# | Наша компания сохраняет атмосферу стартапа, обеспечивая динамичность и стремление к результатам в сочетани...'
ОБРАБОТАННЫЙ ТЕКСТ: 'qa engineer москва м сколково м крылатский getmatch agency python java go c сохранять атмосфера стартап обеспечивать динамичность стремление результат сочетание комфорт стабильность корпоративный среда трудиться создание будущее российский мобильный связь ключевой продукт российский оператор базовый станция lte g разработать основа принцип открытый архитектура open ran приглашать талантливый специалист присоединиться просто сотрудник сторонник разделять страсть прогресс инновация предстоять заниматься участвовать процесс контроль качество разный этап жизненный цикл разрабатывать средство автоматизация функциональный нагрузочный тестирование разрабатывать тестовый документация участвовать лабораторный приёмочный испытание также составлять отчётный 

Preprocessing Train:   0%|          | 0/12500 [00:00<?, ?it/s]

Preprocessing Val:   0%|          | 0/4167 [00:00<?, ?it/s]

In [ ]:
tqdm.pandas(desc="Preprocessing test")
test_df['processed_text'] = test_df['full_text'].progress_apply(preprocess_text)

Preprocessing test:   0%|          | 0/5556 [00:00<?, ?it/s]

## Трансформер


In [ ]:
train_df.to_csv('tran_df_full.csv')

In [ ]:
val_df.to_csv('val_df_full.csv')

In [ ]:
test_df.to_csv('test_df_full.csv')

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset as HFDataset
from sklearn.metrics import r2_score, mean_absolute_error
import numpy as np

In [ ]:
MODEL_NAME = 'ai-forever/ruBert-base'
# sberbank-ai/ruBERT-base, cointegrated/rubert-tiny, ai-forever/ruBert-base
tokenizer   = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_backbone = AutoModel.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/716M [00:00<?, ?B/s]

In [ ]:
# 2. Подготовка данных
# Переименовываем колонки, как того ожидает Trainer
train_bert_df = train_df[['full_text', 'log_salary_from']].rename(columns={'full_text': 'text', 'log_salary_from': 'label'})
val_bert_df = val_df[['full_text', 'log_salary_from']].rename(columns={'full_text': 'text', 'log_salary_from': 'label'})
# test_df = test_df[['full_text']].rename(columns={'full_text': 'text'})

train_hf_dataset = HFDataset.from_pandas(train_bert_df)
val_hf_dataset = HFDataset.from_pandas(val_bert_df)
test_hf_dataset = HFDataset.from_pandas(test_df)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

tokenized_train = train_hf_dataset.map(tokenize_function, batched=True)
tokenized_val = val_hf_dataset.map(tokenize_function, batched=True)
tokenized_test = test_hf_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4167 [00:00<?, ? examples/s]

Map:   0%|          | 0/5556 [00:00<?, ? examples/s]

In [ ]:
class BertRegressor(nn.Module):
    def __init__(self, bert, hid_size=768, dropout=0.2):
        super().__init__()
        self.bert = bert
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.regressor = nn.Sequential(
            nn.Linear(hid_size, hid_size//2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hid_size//2, 1)
        )

    def forward(self, input_ids, attention_mask=None, labels=None):
        last_hidden = self.bert(input_ids, attention_mask).last_hidden_state
        pooled = self.pool(last_hidden.transpose(1, 2)).squeeze(-1)
        logits = self.regressor(pooled).squeeze(-1)           # (B,)

        if labels is not None:                                # train / eval
            loss = F.mse_loss(logits, labels.float())
            return {"loss": loss, "logits": logits}
        else:                                                 # predict / test
            return logits

In [ ]:
model_bert = BertRegressor(bert_backbone).to(device)

In [ ]:
for param in model_bert.bert.parameters():
    param.requires_grad = False


In [ ]:
for name, param in model_bert.named_parameters():
    if param.requires_grad:
        print(f"Градиенты включены: {name}")


Градиенты включены: regressor.0.weight
Градиенты включены: regressor.0.bias
Градиенты включены: regressor.3.weight
Градиенты включены: regressor.3.bias


In [ ]:
from sklearn.metrics import r2_score
import torch.nn.functional as F

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits  # shape (batch,)
    return {"r2": r2_score(labels, preds)}


In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-3,
    num_train_epochs=60,
    warmup_ratio=0.1,
    weight_decay=0.01,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="r2",
    greater_is_better=True,
    report_to="none",
    lr_scheduler_type="cosine"
)


trainer = Trainer(
    model=model_bert,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,R2
1,0.204000,0.203968,0.479922
2,0.218300,0.195741,0.500899
3,0.238700,0.253058,0.354753
4,0.251200,0.251681,0.358264
5,0.259800,0.184205,0.530313
6,0.259400,0.207174,0.471746
7,0.257800,0.208805,0.467588
8,0.262500,0.272511,0.305150
9,0.276800,0.191451,0.511839


KeyboardInterrupt: 

In [ ]:
predictions = trainer.predict(tokenized_test)

In [ ]:
log_preds = np.squeeze(predictions.predictions)

In [ ]:
sub = pd.read_csv('sample_submition.csv')

In [ ]:
sub['prediction'] = log_preds

In [ ]:
sub.to_csv('sub1.csv', index=False)